# Sensitivity Studies

Compare with:

+ Arc
+ Act 1 & 2
+ Demo (Pulsed and SS)

// also adds prototype reactor designs (scylla and charybdis)

## 0.0 Bootup Notebook

In [ ]:
rmprocs(collect(2:Sys.CPU_CORES))
addprocs(Sys.CPU_CORES-1)

@everywhere using Fussy
Main.IJulia.set_current_module(Fussy)

return

## 1.0 Set Defaults

In [ ]:
cur_decks = [
    :scylla, # pulsed
    :charybdis, # steady state
    :arc, :act_1, :act_2,
    :demo_steady, :demo_pulsed
]
  
cur_params = [ 
    :wave_theta, :H, :Q, :epsilon, :kappa_95, :delta_95,
    :nu_n, :nu_T, :l_i, :N_G, :f_D, :Z_eff,
    :max_beta_N, :max_q_95, :max_P_W, :eta_CD,
    :B_CS, :tau_FT
]

return

## 2.0 Define Make Function

In [ ]:
function make_studies(cur_param)
    tmp_studies = Dict()

    for cur_deck in cur_decks
        println(cur_deck)
        
        if cur_deck == :demo_pulsed || cur_deck == :scylla || cur_deck == :demo_steady
            is_consistent = false
        else
            is_consistent = true
        end
        
        cur_study = Study(cur_param, deck=cur_deck, is_consistent=is_consistent, sensitivity=1.0, num_points=41, is_fast=true)
        tmp_studies[cur_deck] = cur_study
        sleep(0.25)
    end

    println("done.")

    tmp_studies
end

return

## 3.0 Get Data

In [ ]:
using FileIO
using JLD2

In [ ]:
has_studies = isdefined(Fussy, :cur_studies)

isdefined(Fussy, :cur_studies) || ( cur_studies = Dict() )

cur_file = "../data/sensitivities.jld2"

if !has_studies && isfile(cur_file)
    has_studies = true
    try
        cur_studies = load(cur_file, "cur_studies")
    catch
        has_studies = false
    end
end

latent_error = nothing
for cur_param in cur_params
    haskey(cur_studies, cur_param) && continue
    println("> ", cur_param, "\n")
    try
        cur_studies[cur_param] = make_studies(cur_param)
    catch cur_error
        latent_error = cur_error
        break
    end
end

isempty(cur_studies) || save(cur_file, Dict("cur_studies" => cur_studies))

( latent_error == nothing ) || rethrow(latent_error)

return

## 4.0 Setup for Plots¶

In [ ]:
cur_fields = [ 
    "T_bar", "n_bar", "I_P", "B_0", "R_CS",
    "P_F", "f_IN", "f_BS", "f_CD", "W_M", "cost", "eta_CD",
    "norm_q_95", "norm_P_E", "norm_P_W", "b", "c", "d"
]

sort!(cur_fields)

unshift!(cur_fields, "R_0")

return

In [ ]:
using Plots

# plotlyjs()

In [ ]:
using Interact
using StringCases
using DataStructures

## 5.0 Make Plot GUI¶

In [ ]:
@manipulate for xvar in cur_params, yvar in cur_fields, deck in cur_decks
    cur_study = cur_studies[xvar][deck]

    work_dict = OrderedDict()
    
    work_dict["Wall"] = cur_study.wall_reactors
    work_dict["Kink"] = cur_study.kink_reactors
    work_dict["Cost"] = cur_study.cost_reactors
    
    filter!(work_reac -> work_reac.is_valid, work_dict["Wall"])
    filter!(work_reac -> work_reac.is_valid, work_dict["Kink"])
    filter!(work_reac -> work_reac.is_valid, work_dict["Cost"])

    max_yy = 0
    
    plot()
    for (cur_index, (cur_key, cur_value)) in enumerate(work_dict)
        isempty(cur_value) && continue
        cur_xx = map(cur_reactor -> getfield(cur_reactor, Symbol(xvar)), cur_value) 
        cur_yy = map(cur_reactor -> getfield(cur_reactor, Symbol(yvar)), cur_value) 

        max_yy = max(max_yy, maximum(cur_yy))
        
        cur_label = ( cur_key == "Cost" ) ? "Cost" : "Beta-$(cur_key)"
        plot!(cur_xx, cur_yy, label=cur_label, color=cur_index)
    end
    plot!()
    
    cur_bot_value = (1-cur_study.sensitivity)*cur_study.default
    cur_top_value = (1+cur_study.sensitivity)*cur_study.default
    
    plot!(
        [cur_study.default,cur_study.default],
        [0,max_yy*1.25], 
        label="Default", style=:dot, color=4, lineopacity=0.55, width=1.5
    )
    
    plot!(
        [cur_bot_value,cur_bot_value,NaN,cur_top_value,cur_top_value],
        [0,max_yy*1.25,NaN,0,max_yy*1.25], 
        label="Edge", style=:dot, color=5, lineopacity=0.55, width=1.5
    )
    
    max_xx = cur_top_value
    
    xlims!(0, max_xx * 1.25)
    ylims!(0, max_yy * 1.25)

    xlabel!(string(xvar))
    ylabel!(yvar)

    cur_title = join(map(capitalize, split(string(deck), "_")), " ")
    title!(cur_title)
end